In [8]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

In [711]:
class Value:
    def __init__(self, data, _children = (), _op = '', label = ''):
        self.data = data
        self._prev = set(_children)
        self.grad = 0.0
        self._backward = lambda:None
        self._op = _op
        self.label = label




    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__ (self, other):
        other = other if isinstance(other,Value) else Value(other)      #assuming othe is eitherint or float
        out = Value((self.data + other.data), (self, other), _op = '+' )
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
    

    def __radd__(self, other): # other + self
        return self + other


    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int and float for now"
        out = Value(self.data ** other, (self, ), f'**{other}')
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value((self.data * other.data), (self, other), _op = '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self*other
    
    def __truediv__(self, other):
        return self * (other)**(-1)

    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)

    def tanh(self):
        n = self.data
        t = (math.exp(2*n) - 1)/(math.exp(2*n)+1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        
        out._backward = _backward
        return out
    



    def backward(self):
        topo = []
        visited= set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [712]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))

        dot.node(name = uid, label= "{ %s| data %.4f | grad %.4f}" % ( n.label, n.data, n.grad), shape ='record')
        if n._op:
            dot.node(name = uid + n._op, label = n._op)

            dot.edge(uid + n._op, uid)
    for n1,n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot        

In [713]:
x1 = Value(2.0 , label= 'x1')
x2 = Value(0.0, label='x2')

w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

b = Value(7.42542354543, label = 'b')

x1w1 = x1 * w1 ; x1w1.label = 'x1w1'
x2w2 = x2 *w2; x2w2.label = 'x2w2'

x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'

n  = x1w1x2w2 + b; n.label = 'n'

#---
e = (2*n).exp(); e.label = 'e'
o = (e-1)/ (e+1); o.label = 'o'
#--- or you can directly do n.tanh()

o.backward()

In [714]:
# draw_dot(o)

In [715]:
class Neuron:       # cell
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        # we want to do w * x + b
        act = sum((wi*xi for wi , xi in zip(self.w, x)), b)
        out = act.tanh()
        return out
    def parameters(self):
        return self.w + [self.b]
    
class Layer:        # tissue
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
        
        
class MLP:          # functioning organ
    def __init__(self, nin, nouts): # nouts will be a list
        sz = [nin] + nouts          # not nouts + [nin]
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [716]:

n = MLP(3, [4,4,1])


In [733]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0,1.0],
    [1.0, 1.0, -1.0],     
]
ys = [1, -1, -1, 1]

xs

[[2.0, 3.0, -1.0], [3.0, -1.0, 0.5], [0.5, 1.0, 1.0], [1.0, 1.0, -1.0]]

In [732]:
for k in range(2000):

    #forward pass
    ypred = [n(x) for x in xs]
    loss = sum(((yout - ygt)**2 for ygt, yout in zip(ys, ypred)))

    #backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    #update
    lr = 1 - 0.001*k
    for p in n.parameters():
        p.data += -1 * lr * p.grad

    print(k, loss.data)

0 7.99997975949832
1 7.9999797529424885
2 7.999979746388966
3 7.999979739837769
4 7.999979733288904
5 7.9999797267423745
6 7.999979720198193
7 7.999979713656371
8 7.999979707116911
9 7.999979700579827
10 7.999979694045125
11 7.9999796875128135
12 7.999979680982901
13 7.999979674455399
14 7.999979667930313
15 7.999979661407652
16 7.999979654887428
17 7.999979648369645
18 7.9999796418543125
19 7.999979635341445
20 7.999979628831045
21 7.999979622323122
22 7.999979615817687
23 7.999979609314746
24 7.999979602814312
25 7.9999795963163916
26 7.99997958982099
27 7.9999795833281215
28 7.999979576837793
29 7.9999795703500105
30 7.999979563864786
31 7.999979557382129
32 7.999979550902044
33 7.999979544424543
34 7.999979537949637
35 7.999979531477329
36 7.999979525007633
37 7.999979518540555
38 7.999979512076104
39 7.99997950561429
40 7.999979499155122
41 7.999979492698607
42 7.999979486244756
43 7.999979479793577
44 7.999979473345078
45 7.99997946689927
46 7.99997946045616
47 7.999979454015758
